https://www.youtube.com/watch?v=kCc8FmEb1nY

In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
# Read dataset
path_in = "data/shakespeare.txt"
with open(path_in, 'r') as fh:
    text = fh.read().strip()
len(text), text[:1000]

(1115393,
 "First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this

In [4]:
# Tokenizer
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda text: [stoi[ch] for ch in text]
decode = lambda ids: ''.join(itos[i] for i in ids)
encode("hello world"), decode(encode("hello world"))

([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42], 'hello world')

In [5]:
# To tensor
data = torch.tensor(encode(text), dtype=torch.long)
data.shape, data.dtype, data[:100]

(torch.Size([1115393]),
 torch.int64,
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
         53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
          1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
         57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
          6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
         58, 47, 64, 43, 52, 10,  0, 37, 53, 59]))

In [6]:
# Subsets split
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]
len(train_data), len(val_data)

(1003853, 111540)

In [7]:
# Batching
torch.manual_seed(42)
block_size = 8 # max context length
batch_size = 4 # max parallely processed examples

def get_batch(split):
    data = train_data if split == 'train' else val_data
    starts = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in starts])
    y = torch.stack([data[i+1:i+block_size+1] for i in starts])
    return x, y

batch_x, batch_y = get_batch('train')
batch_x.shape, batch_y.shape, batch_x, batch_y

(torch.Size([4, 8]),
 torch.Size([4, 8]),
 tensor([[54, 43, 39, 49,  1, 39, 45, 39],
         [43, 56, 57, 11,  1, 61, 46, 53],
         [43,  1, 46, 39, 58, 46,  1, 58],
         [43,  1, 47, 57,  1, 57, 53,  1]]),
 tensor([[43, 39, 49,  1, 39, 45, 39, 47],
         [56, 57, 11,  1, 61, 46, 53,  6],
         [ 1, 46, 39, 58, 46,  1, 58, 56],
         [ 1, 47, 57,  1, 57, 53,  1, 50]]))

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None): # idx.shape == targets.shape == batch_size x block_size [4, 8]
        logits = self.token_embedding_table(idx) # tensor batch_size x block_size x vocab_size [4, 8, 65] ~ [Batch, Time, Channel]
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy, negative log likelihood
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # [B,T,C] -> [B,C] for last Time step
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1) # [B,C] -> [B,1]; take 1 random C for each B
            idx = torch.cat((idx, idx_next), dim=1) # [B,T+1]
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(batch_x, batch_y)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long) # 0 == \n
decode(m.generate(idx, max_new_tokens=100)[0].tolist())


torch.Size([32, 65])
tensor(4.7187, grad_fn=<NllLossBackward0>)


"\nuoiaF$z\nM?kI;h\nDbuMG,H3LYNmrDxKgTpvAKOF-jU.hc;fBMTGa-IS\ng3lEb&ZQ,l;:m;lpcNN\nKpVEYRIIM,'hCRbMAcWTkrnH"

In [9]:
# Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [10]:
# Estimate loss
eval_iters = 200

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out



In [22]:
# Training loop
batch_size = 32
for i in range(10_000):

    if i % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {i:8}, train loss: {losses['train']:8.5f}, eval loss: {losses['val']:8.5f}")

    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

Step:        0, train loss:  2.46986, eval loss:  2.48891
Step:      200, train loss:  2.46441, eval loss:  2.48163
Step:      400, train loss:  2.47061, eval loss:  2.48342
Step:      600, train loss:  2.46372, eval loss:  2.47555
Step:      800, train loss:  2.46449, eval loss:  2.48814
Step:     1000, train loss:  2.46199, eval loss:  2.48468
Step:     1200, train loss:  2.45904, eval loss:  2.48919
Step:     1400, train loss:  2.45831, eval loss:  2.47194
Step:     1600, train loss:  2.45307, eval loss:  2.47909
Step:     1800, train loss:  2.46323, eval loss:  2.48977
Step:     2000, train loss:  2.46453, eval loss:  2.47476
Step:     2200, train loss:  2.45728, eval loss:  2.47103
Step:     2400, train loss:  2.45310, eval loss:  2.47943
Step:     2600, train loss:  2.44857, eval loss:  2.48196
Step:     2800, train loss:  2.46144, eval loss:  2.49010
Step:     3000, train loss:  2.46358, eval loss:  2.48035
Step:     3200, train loss:  2.45377, eval loss:  2.48171
Step:     3400

In [11]:
decode(m.generate(idx, max_new_tokens=500)[0].tolist())

"\nDKVam&K3ZnkIMaqntjh-AW'BJxxghjZTBCnCU?qBQbwyCjhRhSLyJfGtsutJBy-j&,r,'k$PTuZm3tPbu NwH-IPeaquJVrK'x fK-d3si'g.cu,k-JMiv?R-jlqIeNiaqan\n3VEmtWcEIcO!bAFDHhSsYzEn;UGI;f .bYh,g:gSOr:s3axTBykJG.!gsz ffgpGLHa3xDkWlrwVm3C\nKh.H?Hvy!ogQahk!DmgTZ&X.SkmWdkovk?qKpjvTWPSJNzB?qn usMXFU.cYFAWVkJI-JP\nLwmLBgBNiyZuLietbGFfUN?AFDINUBGiZBtKpt;LuoERDPImKHh!TwVTyy.lWC:ZKp-r!MV!vVoivkX yUROKGczW'scCR;QN?umaTozJCfhSnDc.cJnQWkkqIaqfSesGnx'dO;cdC YVeYRpIYwcGPeuv.fBy:mMKTwXs-It,XhCh.;$oblc- YpqgB?-jhSkJMAFq\n3sn';,blGtY?xNwT"

In [30]:
# Tensor reshape - eg. flatten
t = torch.tensor([[1,2,3], [4,5,6]], dtype=torch.int)
t.view(3, 2), t.view(-1, 1, 1)

(tensor([[1, 2],
         [3, 4],
         [5, 6]], dtype=torch.int32),
 tensor([[[1]],
 
         [[2]],
 
         [[3]],
 
         [[4]],
 
         [[5]],
 
         [[6]]], dtype=torch.int32))

In [12]:
# Torch for attention
torch.manual_seed(42)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# Compute normalized average of all past tokens
# context V1 loop
# iterate over batches in time steps, keep all channels
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        x_prev = x[b, :t+1] # [t,C], take single b, all past t's and the entire C
        xbow[b, t] = torch.mean(x_prev, 0) # 0 = t axis, 1 = C axis
xbow.shape, x[0], xbow[0]

(torch.Size([4, 8, 2]),
 tensor([[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [ 0.6784, -1.2345],
         [-0.0431, -1.6047],
         [-0.7521,  1.6487],
         [-0.3925, -1.4036],
         [-0.7279, -0.5594],
         [-0.7688,  0.7624]]),
 tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]]))

In [21]:
# context V2 matrix multiplication
# upper triangular masking
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # upper triangular masking
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(f"{a=}")
print(f"{b=}")
print(f"{c=}")

a=tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [22]:
# normalize
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(f"{a=}")
print(f"{b=}")
print(f"{c=}")

a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [34]:
wei = torch.tril(torch.ones(T, T)) # [T,T]
wei = wei / torch.sum(wei, 1, keepdim=True) # [T,T]
xbow2 = wei @ x # [T,T] @ [B,T,C] -> can't multiply
# so torch will auto expand to B to match dim0=B
# batched matrix multiply
# [B,T,T] @ [B,T,C] -> [B,T,C]
xbow2.shape, torch.allclose(xbow, xbow2)


(torch.Size([4, 8, 2]), True)

In [38]:
# context V3 softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float('-inf')) # == cut out future tokens
wei = F.softmax(wei, dim=-1) # normalization
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [45]:
# context V4 self attention
torch.manual_seed(42)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C) # token private information

# Single head
head_size = 16
query = nn.Linear(C, head_size, bias=False) # what am i looking for (autodetected pattern)
key = nn.Linear(C, head_size, bias=False)  # what do I contain 
# if key and query are aligned, they create HIGH AFFINITY = interact to a high amount and they will cotribute more for current token as opposed to other tokes in the sequence
value = nn.Linear(C, head_size, bias=False)  # filter for particular attention head

# forward modules by x; parallel = no communication so far
q = query(x) # [B,T,head_size]
k = key(x)   # [B,T,head_size]

# communincate:
wei = q @ k.transpose(-2, -1) # [B,T,head_size] @ [B,head_size, T] -> [B,T,T] = for each batch, take learned time interactions between tokens
wei = wei * head_size**-0.5 # scale attention - lessen the values so that softmax won't go to extremely low values ("converge towards one-hot vectors")

tril = torch.tril(torch.ones(T,T)) # B dim broadcasting
# before wei was just zeros, now it's attention matrix
# wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float('-inf')) # we want or want not to discard future tokens, depeding on task
wei = F.softmax(wei, dim=-1) # boosts high affinity

v = value(x)
out = wei @ v
out.shape, out[0]


(torch.Size([4, 8, 16]),
 tensor([[ 0.7630, -0.2412, -0.4150,  0.3833,  0.5740, -1.6738,  0.7954,  0.6872,
          -0.3848,  0.5073, -0.5312, -0.1221,  0.0445,  1.2169,  0.9940,  1.5281],
         [ 0.4894, -0.1269, -0.6833, -0.3046,  0.2735, -0.7042,  0.2440, -0.3779,
          -0.1338,  0.3596,  0.1304,  0.0420,  0.1211,  0.6841,  0.5222,  0.2674],
         [ 0.2229,  0.0217, -0.2720,  0.1393,  0.5338, -0.7021,  0.1438, -0.1945,
          -0.3118,  0.3121,  0.0133, -0.1628, -0.3413,  0.5239,  0.7935,  0.6615],
         [ 0.1464,  0.1534, -0.2507, -0.0095,  0.1002, -0.5184, -0.0287, -0.5144,
          -0.2047,  0.2677,  0.0913,  0.1241,  0.1922,  0.5099,  0.3388,  0.2016],
         [ 0.0442,  0.1569, -0.2724,  0.2373,  0.2169, -0.4894, -0.2219, -0.3260,
          -0.0747,  0.1438, -0.0729,  0.0190, -0.0435,  0.3857,  0.3784,  0.2121],
         [ 0.0259,  0.0879, -0.2091,  0.1253,  0.2116, -0.5392, -0.1011, -0.2242,
          -0.1561,  0.1605,  0.0565,  0.0393,  0.0089,  0.3979,  0.3

Attention is 
- a communication method
- a weighted sum of focus

Self attention 
- same input x creates queries, keys and values

Cross attention
- different source of data - keys + values, own queries

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None): # idx.shape == targets.shape == batch_size x block_size [4, 8]
        logits = self.token_embedding_table(idx) # tensor batch_size x block_size x vocab_size [4, 8, 65] ~ [Batch, Time, Channel]
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy, negative log likelihood
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # [B,T,C] -> [B,C] for last Time step
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1) # [B,C] -> [B,1]; take 1 random C for each B
            idx = torch.cat((idx, idx_next), dim=1) # [B,T+1]
        return idx

